In [29]:
from scipy.stats import poisson
import numpy as np
import warnings

def truncPois_CDF(y, mu, log=True):
    is_scalar = np.isscalar(y)

    if is_scalar:
        y = np.array([y])

    f_zero = poisson.pmf(0, mu)

    if log:
        log_cdf_y = np.log((poisson.cdf(y, mu) - poisson.cdf(0, mu)) / (1 - f_zero))
        log_cdf_y[y <= 0] = float('-inf')

        if is_scalar:
            return log_cdf_y[0]
        else:
            return log_cdf_y
    else:
        cdf_y = (poisson.cdf(y, mu) - poisson.cdf(0, mu)) / (1 - f_zero)
        cdf_y[y <= 0] = 0

        if is_scalar:
            return cdf_y[0]
        else:
            return cdf_y
        
def log1mexp(x):
    if np.any((x < 0) & (~np.isnan(x))):
        raise ValueError("Inputs need to be non-negative!")
    return np.where(x <= np.log(2), np.log(-np.expm1(-x)), np.log1p(-np.exp(-x)))
        

#log.p	logical; if TRUE, probabilities p are given as log(p)    
def qpois_trunc(p, lam, log_p=False):
    # if f(0)=0 no truncation is needed
    if poisson.pmf(0, lam) <= 1e-9:
        print('Abkürzung')
        return poisson.ppf(p, lam)
    else:
        # Convert p to array if it's a single value
        if not isinstance(p, (list, np.ndarray)):
            p = np.array([p])
        
        # Set log-probabilities (lower tail)
        n = len(p)
        if log_p:
            logp = p
        else:
            logp = np.log(p)
        
        # Set output and deal with special cases (outputs NA and Inf)
        quantiles = np.full(n, np.nan)
        nna = ~np.isnan(logp)
        nlogp = logp[nna]
        if len(nlogp) == 0:
            return quantiles
        
        quantiles[nna] = np.full(len(nna), np.inf)
        if np.min(nlogp) >= 0:
            return quantiles

        # Set log-CDF vector
        lp_max = np.max(nlogp[nlogp < 0])

        # find an adequate upper limit, starting from the extreme conservative chebychev inequality
        upper = int(lam + np.sqrt(lam * np.exp(-log1mexp(-lp_max)))) #Chebychev inequality

        # if upper < 1000 there is an log(0)=-inf with warning -> ignore this warning
        warnings.filterwarnings('ignore', category=RuntimeWarning)

        while np.exp(truncPois_CDF(upper-1000, lam)) > 0.999:
            upper = upper - 1000

        # after thsi section warnings are enabled again
        warnings.filterwarnings('default', category=RuntimeWarning)

        yarray = np.arange(1, int(upper)+1)
        logcdf = truncPois_CDF(yarray, lam)

        # Compute output
        for i in range(n):
            if nna[i]:
                if logp[i] < 0:
                    quantiles[i] = np.sum(logcdf < np.array(logp[i])) + 1 #+1 because 0 is truncated
        
        # Return output
        if len(quantiles) == 1:
            return quantiles[0]
        else:
            return quantiles

In [37]:
# calculation of quantiles
quantiles = np.arange(0.001, 0.9999, 0.001)
quantiles = np.round(quantiles, 3)

lam = 6

a = poisson.ppf(quantiles, lam)
b = qpois_trunc(quantiles, lam)

In [38]:
a

array([ 0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4

In [39]:
b

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4

In [40]:
a == b

array([False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,